## 事前準備

必要な依存関係のインポートや定数の定義をします

In [ ]:
from pandas import *
from pylab import *
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from matplotlib.gridspec import GridSpec
from soracom_client import SoracomClient

## SORACOM APIを呼び出して各種データを取得

SIM一覧、トラフィック、エラーログなどを取得します

In [ ]:
sc = SoracomClient()
duration_days = 14
subscribers = sc.getSubscribers()
traffic = sc.getAirStatsByDay(duration_days)
logs = sc.getLogs(duration_days)

## レポートを描画

各種メトリクスと、エラーログを描画します。
エラーログについては若干乱暴ですが、DataFrameをそのまま表示します。（便利なので）

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(15,15))
gs = GridSpec(4,3, figure=fig)

#ax = axes[0,0]
ax = fig.add_subplot(gs[0,0])
ax.set_title('# of SIM by status')
subscribers.groupby(['status'])['imsi'].count().plot.bar(ax=ax)

#ax = axes[0,1]
ax = fig.add_subplot(gs[0,1])
ax.set_title('# of SIM by speed class')
subscribers.groupby(['speedClass'])['imsi'].count().plot.bar(ax=ax)

#ax = axes[0,2]
ax = fig.add_subplot(gs[0,2])
ax.set_title('# of SIM by online')
subscribers.groupby(['online'])['imsi'].count().plot.bar(ax=ax)

ax = fig.add_subplot(gs[1,:])
traffic.groupby(['dates'])[[
    'downloadByteSizeTotal',
    'uploadByteSizeTotal'
]] \
    .sum().plot(
        ax=ax,
        title="Traffic usage by day"
    )

ax = fig.add_subplot(gs[2,:])
topNdownload = traffic.groupby(['imsi'])['downloadByteSizeTotal'] \
    .sum().head(10).index.tolist()
traffic[traffic['imsi'].isin(topNdownload)] \
    .groupby(['dates','name'])['downloadByteSizeTotal'] \
    .sum().unstack().plot(
        ax=ax,
        title="Top 10 download by SIM"
    )

ax = fig.add_subplot(gs[3,:])
topNupload = traffic.groupby(['imsi'])['uploadByteSizeTotal'] \
    .sum().head(10).index.tolist()
traffic[traffic['imsi'].isin(topNdownload)] \
    .groupby(['dates','name'])['uploadByteSizeTotal'] \
    .sum().unstack().plot(
        ax=ax,
        title="Top 10 upload by SIM"
    )

plt.show()

In [ ]:
logs[['service','resourceType','resourceId','logLevel','message']]